# EX 3 -
In this exercise you will implement your first word recognizer using the KNN
algorithm. Your algorithm will recognize the digits 1 - 5. For that, You will
implement the KNN algorithm, where the distance metric should be the L2.

In [3]:
!pip install librosa

  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201381 sha256=a02f2bfa2a15449127c00c7f3f72477c1d95fec9d00a9105afc2222531d8726f
  Stored in directory: c:\users\yoav_\appdata\local\pip\cache\wheels\de\1e\aa\d91797ae7e1ce11853ee100bee9d1781ae9d750e7458c95afb
  Created wheel for audioread: filename=audioread-2.1.8-py3-none-any.whl size=23095 sha256=a6c58b8e7f4fd99dcd30138f831cf68f3e2aefc7ac274ec92cd143743553d539
  Stored in directory: c:\users\yoav_\appdata\local\pip\cache\wheels\cd\85\04\704a589ef8773d1ec1393c4b271a023f929d85ef5317b12d8c
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320723 sha256=b71e5b152a9013d38ae2566bffc57b22ff2769782683f589edfbf45346c302ff
  Stored in directory: c:\users\yoav_\appdata\local\pip\cache\wheels\a0\18\0a\8ad18a597d8333a142c9789338a96a6208f1198d290ece356c
Successfully built librosa audioread resampy


In [4]:
import numpy as np
import librosa 
import glob
from scipy import stats



In [5]:
def readWAV(root_folder):
    """Read the .wav files from directory.
        root folder: is the name of the the directory locate.
        Return: list of paths for all the files."""

    wav_files=[]
    if(root_folder=="train_data"):
        for folder in glob.glob("ex3\\*\\*\\*.wav"):
            wav_files.append((getLabel(str(folder)),str(folder)))
    else:
        for folder in glob.glob("ex3\\*\\*.wav"):
            wav_files.append((getLabel(str(folder)),str(folder)))
    return wav_files 


In [6]:
def getLabel(path):
    """ The method is replaces the word with the number itself.
        The word is part of the path."""
    if("five" in path):
        return "5"
    if("four" in path):
        return "4"
    if("one" in path):
        return "1"
    if("three" in path):
        return "3"
    if("two" in path):
        return "2"
    return ""


In [7]:
def get_ALL(root_folder):
    """The method get the root folder and matrix for each file:
        [(label,file path), mfcc for the .wav file]
        """
    tempL=readWAV(root_folder) #list of all paths
    mfccMAT=[]
    for file in tempL:
        label=file[0]
        file_path=file[1]
        mfccMAT.append(((label,file_path),getMFCC(file_path)))
    
    return mfccMAT


In [8]:
def calculateDistance(train_data,test_data,test):
    """The method calculate a euclidean distance for each test file aginst all the training data.
        It create list sorted by euclidean distance. 
        train_data - list of files and mfcc of them 
        tset_data - one sample of test data
        test - *optinal* test file name"""
    distance=[]
    for train in train_data:
        label=train[0][0]
        path=train[0][1]
        distance.append(((label,path.split("\\")[-1]),(test,norm(train[1],test_data))))
    distance = sorted(distance,key=lambda x:x[1][1])
    return distance


In [9]:
def getMFCC(f_path):
    y,sr=librosa.load(f_path,sr=None)
    mfcc=librosa.feature.mfcc(y=y,sr=sr)
    mfcc= stats.zscore(mfcc,axis=1)
    return mfcc
   

In [10]:
def norm(a,b):
    return np.linalg.norm(a-b)
    

In [11]:
from collections import Counter

def most_common(lst):
    data = Counter(lst)
    return max(lst, key=data.get)

def getKNNresults(norm,k):
    """The method find the most fre """
    results=[]
    l=[]
    for sample in norm:
        l=[]
        sentence=""
        for i in range(k):
            l.append(int(sample[1][i][0][0]))
        sentence=f"{sample[0]} - {most_common(l)}"
        results.append(sentence)

        
    return results
   

In [12]:
def accuracy(knn_result,actual_labels):
    totalCount=0
    catchCount=0
    for label in knn_result:
        file_name,file_label=str(label).rsplit(".wav",1)
        file_name=file_name+".wav"
        realValue=actual_labels[f"{file_name} "] #get the real label taged for file
        if(int(realValue)!=0):#label isn't tag as 0
            totalCount+=1
            if("Label was not decided" not in file_label ):
                if(int(file_label[-1])==int(realValue)):
                    catchCount+=1
    return(f"total labels {totalCount} \n matching labels {catchCount}")


In [13]:
def getActualLabels():
    acutalLabels=[]
    with open("prediction.txt","r") as file:
        acutalLabels=file.read()
    acutalLabels=acutalLabels.split("\n")
    dicLabels={}
    for label in acutalLabels:
        file_name,file_label=str(label).rsplit("-",1)
        dicLabels[file_name]=file_label
    return(dicLabels)


In [16]:
def main():  
    
    train_data=get_ALL("train_data")
    test_data=readWAV("test_data")
    euclidean_distance=[]
    knn=5

    for test in test_data:
        euclidean_distance.append((test[1].split("\\")[-1], 
                                   calculateDistance(train_data, 
                                                     getMFCC(test[1]), 
                                                     test)))

    l=getKNNresults(euclidean_distance,knn)

    with open("output.txt","w+") as file:
        file.write("\n".join(str(item) for item in l))
        file.write("\n____________________ \n")
        file.write((str(accuracy(l, getActualLabels()))))
        
if(__name__ == "main"):
    main()
